In [187]:
import pandas as pd
import numpy as np
import pylab as pl
from sklearn import linear_model
from sklearn import metrics
import statsmodels.formula.api as smf
import os
import sys
import geopandas as gp

% pylab inline

Populating the interactive namespace from numpy and matplotlib


In [238]:
merge2 = pd.DataFrame.from_csv('Merged_V2.csv')

In [189]:
merge2.columns

Index([u'PrecinctYear', u'Precinct', u'Year', u'Taxis', u'Flyers', u'Seniors',
       u'Outreach', u'HandsOn', u'Edu_Total', u'Backing Unsafely',
       u'Brake Lights (Defect.or Improper)', u'Bus Lane, Driving in',
       u'Cell Phone', u'Commercial Veh on Pkwy', u'Defective Brakes',
       u'Disobey Steady Red Signal', u'Disobey Traffic Control Device',
       u'Equipment (Other)', u'Fail to Keep Right', u'Fail to Signal',
       u'Following Too Closely', u'Headlights (Defect. or Improper)',
       u'Improper Lights', u'Improper Passing', u'Improper Turn',
       u'Improper/Missing Plates', u'Not Giving R of W to Pedes.',
       u'Not Giving R of W to Veh.', u'One Way Street', u'Other Movers',
       u'Pavement Markings', u'Safety Belt', u'School Bus, Passing Stopped',
       u'Speeding', u'Spillback', u'Tinted Windows', u'Truck Routes',
       u'U-Turn', u'Uninspected', u'Uninsured', u'Unlicensed Operator',
       u'Unregistered', u'P12', u'P123', u'Traf_Total',
       u'NUMBER OF P

In [190]:
street_design = merge2.drop([u'Taxis', u'Flyers', u'Seniors',
       u'Outreach', u'HandsOn', u'Edu_Total', u'Backing Unsafely',
       u'Brake Lights (Defect.or Improper)', u'Bus Lane, Driving in',
       u'Cell Phone', u'Commercial Veh on Pkwy', u'Defective Brakes',
       u'Disobey Steady Red Signal', u'Disobey Traffic Control Device',
       u'Equipment (Other)', u'Fail to Keep Right', u'Fail to Signal',
       u'Following Too Closely', u'Headlights (Defect. or Improper)',
       u'Improper Lights', u'Improper Passing', u'Improper Turn',
       u'Improper/Missing Plates', u'Not Giving R of W to Pedes.',
       u'Not Giving R of W to Veh.', u'One Way Street', u'Other Movers',
       u'Pavement Markings', u'Safety Belt', u'School Bus, Passing Stopped',
       u'Speeding', u'Spillback', u'Tinted Windows', u'Truck Routes',
       u'U-Turn', u'Uninspected', u'Uninsured', u'Unlicensed Operator',
       u'Unregistered', u'P12', u'P123', u'Traf_Total'], axis = 1)

In [191]:
street_design.head()

PrecinctYear  Precinct  Year  NUMBER OF PERSONS INJURED  \
0        682013      68.0  2013                        588   
1        682014      68.0  2014                        615   
2        682015      68.0  2015                        533   
3       1022013     102.0  2013                        788   
4       1022014     102.0  2014                        813   

   NUMBER OF PERSONS KILLED  NUMBER OF PEDESTRIANS INJURED  \
0                         4                            117   
1                         4                            127   
2                         3                             98   
3                         6                            139   
4                         5                            134   

   NUMBER OF PEDESTRIANS KILLED  NUMBER OF CYCLIST INJURED  \
0                             3                         37   
1                             3                         33   
2                             0                         34   
3                             5                         63   
4                             1                         39   

   NUMBER OF CYCLIST KILLED  NUMBER OF MOTORIST INJURED  \
0                         0                         434   
1                         0                         455   
2                         0                         401   
3                         0                         586   
4                         1                         640   

   NUMBER OF MOTORIST KILLED  Neigh_Slow_Zone_Total  Speed_Bump_Total  
0                          1                    NaN               1.0  
1                          1                    NaN               2.0  
2                          3                    NaN               3.0  
3                          1                    NaN               2.0  
4                          3                    NaN               8.0

In [192]:
# na values were created when an area without any speed bumps was added therefore fill with 0
street_design.fillna(0, inplace = True)

In [193]:
mean_injured = street_design['NUMBER OF PERSONS INJURED'].mean()
std_injured = street_design['NUMBER OF PERSONS INJURED'].std()
mean_killed = street_design['NUMBER OF PERSONS KILLED'].mean()
std_killed = street_design['NUMBER OF PERSONS KILLED'].std()

In [194]:
street_design['standardized_injuries'] = [(x - mean_injured) / std_injured for x in street_design['NUMBER OF PERSONS INJURED']]
street_design['standardized_death'] = [(x - mean_killed) / std_killed for x in street_design['NUMBER OF PERSONS KILLED']]

In [195]:
injury_change = []
injury_change_2014 = street_design.groupby('Precinct')['standardized_injuries'].pct_change()
injury_change_2015 = street_design.groupby('Precinct')['standardized_injuries'].pct_change(periods = 2)
for x in range(len(street_design)):
    if street_design.iloc[x, 2] == 2013.0: 
        injury_change.append(0)
    elif street_design.iloc[x, 2] == 2014.0:
        injury_change.append(injury_change_2014[x])
    else:
        injury_change.append(injury_change_2015[x])
street_design['standardized_injuries_change'] = injury_change

In [196]:
proj_corr = pd.DataFrame.from_csv('street_improvement_project_corridors')
proj_inter = pd.DataFrame.from_csv('street_improvement_project_intesections')

In [197]:
proj_corr.head()

precinct  Corridor_Proj_Total                         type
0       1.0                    0          Bicycle Network Dev
1       1.0                   15  Bicycle Network Development
2       1.0                    0             Bike Network Dev
3       1.0                    0          High Crash Corridor
4       1.0                    0             New Public Space

In [198]:
pivoted = pd.pivot_table(proj_corr, index = 'precinct', columns = 'type', values = 'Corridor_Proj_Total', aggfunc = sum)

In [199]:
pivoted['Bikes'] = pivoted['Bicycle Network Dev'] + pivoted['Bicycle Network Development'] + pivoted['Bike Network Dev']

In [200]:
pivoted.drop(['Bicycle Network Dev', 'Bicycle Network Development', 'Bike Network Dev'], axis = 1, inplace = True)

In [201]:
pivoted.columns = ['High_Crash_Corridor_c', u'New_Public_Space_c', u'Pedestrian_Safety_c',
       u'Senior_Safety_c', u'Traffic_Calming_c', u'Traffic_Network_Chng_c',
       u'VZ_Priority_Geo_c', u'Vision_Zero_c', u'Bikes_c']

In [202]:
pivoted2 = pd.pivot_table(proj_inter, index = 'precinct', columns = 'type', values = 'Intersec_Proj_Total', aggfunc = sum)

In [203]:
pivoted2.columns = ['High_Crash_Corridor_i', u'Pedestrian_Safety_i', u'Senior_Safety_i',
                   u'Traffic_Calming_i', u'Traffic_Network_Chng_i', u'VZ_Priority_Geo_i',
                   u'Vision_Zero_i']

In [204]:
with_projects = pd.merge(street_design, pivoted, left_on = 'Precinct', right_index = True, how = 'outer')

with_projects = pd.merge(with_projects, pivoted2, left_on = 'Precinct', right_index = True, how = 'outer')

with_projects.fillna(0, inplace = True)

In [205]:
data = with_projects[with_projects.Year > 2013]

In [206]:
eqt = ['Speed_Bump_Total', 'Neigh_Slow_Zone_Total', 'High_Crash_Corridor_c', 'New_Public_Space_c',
        'Pedestrian_Safety_c', 'Senior_Safety_c', 'Traffic_Calming_c', 'Traffic_Network_Chng_c',
        'VZ_Priority_Geo_c', 'Vision_Zero_c', 'Bikes_c', 'High_Crash_Corridor_i',
        'Pedestrian_Safety_i', 'Senior_Safety_i', 'Traffic_Calming_i', 'Traffic_Network_Chng_i',
        'VZ_Priority_Geo_i', 'Vision_Zero_i' ]
eqt = 'standardized_injuries_change ~ ' + ' + '.join(eqt) + ' - 1'

In [207]:
eqt

'standardized_injuries_change ~ Speed_Bump_Total + Neigh_Slow_Zone_Total + High_Crash_Corridor_c + New_Public_Space_c + Pedestrian_Safety_c + Senior_Safety_c + Traffic_Calming_c + Traffic_Network_Chng_c + VZ_Priority_Geo_c + Vision_Zero_c + Bikes_c + High_Crash_Corridor_i + Pedestrian_Safety_i + Senior_Safety_i + Traffic_Calming_i + Traffic_Network_Chng_i + VZ_Priority_Geo_i + Vision_Zero_i - 1'

In [208]:
lm = smf.ols(eqt, data).fit()

In [209]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     standardized_injuries_change   R-squared:                       0.117
Model:                                      OLS   Adj. R-squared:                 -0.000
Method:                           Least Squares   F-statistic:                    0.9974
Date:                          Mon, 05 Dec 2016   Prob (F-statistic):              0.467
Time:                                  14:29:21   Log-Likelihood:                -263.38
No. Observations:                           154   AIC:                             562.8
Df Residuals:                               136   BIC:                             617.4
Df Model:                                    18                                         
Covariance Type:                      nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Speed_Bump_Total          -0.0113      0.022     -0.507      0.613        -0.056     0.033
Neigh_Slow_Zone_Total     -0.2605      0.343     -0.760      0.448        -0.938     0.417
High_Crash_Corridor_c     -0.0264      0.029     -0.917      0.361        -0.083     0.031
New_Public_Space_c         0.0799      0.252      0.317      0.752        -0.419     0.578
Pedestrian_Safety_c       -0.0747      0.046     -1.610      0.110        -0.166     0.017
Senior_Safety_c            0.0345      1.026      0.034      0.973        -1.995     2.064
Traffic_Calming_c         -0.0069      0.009     -0.760      0.449        -0.025     0.011
Traffic_Network_Chng_c    -0.0560      0.065     -0.862      0.390        -0.185     0.073
VZ_Priority_Geo_c          0.0074      0.007      1.003      0.317        -0.007     0.022
Vision_Zero_c             -0.0143      0.015     -0.931      0.354        -0.045     0.016
Bikes_c                    0.1174      0.046      2.578      0.011         0.027     0.207
High_Crash_Corridor_i      0.0631      0.102      0.618      0.538        -0.139     0.265
Pedestrian_Safety_i        0.1051      0.138      0.764      0.446        -0.167     0.377
Senior_Safety_i            0.3270      1.392      0.235      0.815        -2.426     3.080
Traffic_Calming_i          0.1152      0.147      0.786      0.433        -0.175     0.405
Traffic_Network_Chng_i    -1.1872      0.671     -1.770      0.079        -2.514     0.139
VZ_Priority_Geo_i          0.2419      0.157      1.544      0.125        -0.068     0.552
Vision_Zero_i             -0.0021      0.204     -0.010      0.992        -0.406     0.402
==============================================================================
Omnibus:                       51.426   Durbin-Watson:                   1.896
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              720.862
Skew:                           0.678   Prob(JB):                    2.93e-157
Kurtosis:                      13.512   Cond. No.                         349.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [210]:
X = pd.concat([with_projects['Speed_Bump_Total'], with_projects['Neigh_Slow_Zone_Total'], with_projects.iloc[:, -16:]], axis = 1)
y = with_projects['standardized_injuries_change']

In [211]:
lasso = linear_model.Lasso(fit_intercept = False, alpha = 1000).fit(X, y)

In [212]:
lasso.coef_

array([-0., -0., -0.,  0., -0.,  0., -0., -0., -0., -0.,  0., -0.,  0.,
       -0., -0., -0.,  0., -0.])

In [213]:
precincts = gp.GeoDataFrame.from_file(os.getenv('HOME') + '/Applied_Data_Science/VZ_data/Precincts/summary_police_precincts.shp')

In [214]:
precincts_shp = precincts.drop([u'Ages_10_to', u'Ages_15_to', u'Ages_18_an', u'Ages_20_to',
       u'Ages_25_to', u'Ages_45_to', u'Ages_5_to_', u'Ages_65_an',
       u'Ages_Under',    u'SUM_ASZ', u'SUM_BikeFa',
       u'SUM_BikeIn', u'SUM_Fatali', u'SUM_Injuri',    u'SUM_LPI',
       u'SUM_MVOFat', u'SUM_MVOInj', u'SUM_NeighS', u'SUM_PedFat',
       u'SUM_PedInj', u'SUM_SIPCor', u'SUM_SIPInt', u'SUM_Signal',
       u'SUM_SpeedH', u'SUM_VZ_P_1', u'SUM_VZ_P_2', u'SUM_VZ_Pri'], axis = 1)

In [215]:
precincts_shp.Shape_Area = precincts_shp.Shape_Area / (2.788 * 10 ** 7) # convert shape area to miles

In [216]:
precincts_shp.Precint = precincts_shp.Precinct.astype(float)

In [217]:
with_projects = pd.merge(with_projects, precincts_shp, left_on = 'Precinct', right_on = 'Precinct')

In [218]:
with_projects['Per_Cap_Injury'] = with_projects['NUMBER OF PERSONS INJURED'] / (with_projects['Total_Pop'] / 10000)

In [219]:
eqt = ['Speed_Bump_Total', 'Neigh_Slow_Zone_Total', 'High_Crash_Corridor_c', 'New_Public_Space_c',
        'Pedestrian_Safety_c', 'Senior_Safety_c', 'Traffic_Calming_c', 'Traffic_Network_Chng_c',
        'VZ_Priority_Geo_c', 'Vision_Zero_c', 'Bikes_c', 'High_Crash_Corridor_i',
        'Pedestrian_Safety_i', 'Senior_Safety_i', 'Traffic_Calming_i', 'Traffic_Network_Chng_i',
        'VZ_Priority_Geo_i', 'Vision_Zero_i' ]
eqt = 'Per_Cap_Injury ~ ' + ' + '.join(eqt) + ' - 1'

In [220]:
data2 = with_projects[with_projects.Total_Pop > 0][with_projects.Year > 2013] # one precinct was opened in 2013 and odes not have demog data
lm2 = smf.ols(eqt, data = data2).fit()

/opt/rh/anaconda/root/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [221]:
lm2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         Per_Cap_Injury   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                 -0.094
Method:                 Least Squares   F-statistic:                    0.2763
Date:                Mon, 05 Dec 2016   Prob (F-statistic):              0.999
Time:                        14:29:22   Log-Likelihood:                -1463.0
No. Observations:                 152   AIC:                             2962.
Df Residuals:                     134   BIC:                             3017.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Speed_Bump_Total          36.5138     61.378      0.595      0.553       -84.882   157.910
Neigh_Slow_Zone_Total   -247.0915    940.095     -0.263      0.793     -2106.436  1612.253
High_Crash_Corridor_c     -5.9095     78.864     -0.075      0.940      -161.889   150.070
New_Public_Space_c        -1.9200    690.707     -0.003      0.998     -1368.017  1364.177
Pedestrian_Safety_c      279.9338    127.257      2.200      0.030        28.242   531.625
Senior_Safety_c         -338.0461   2812.863     -0.120      0.905     -5901.398  5225.306
Traffic_Calming_c          4.8453     24.838      0.195      0.846       -44.280    53.971
Traffic_Network_Chng_c    39.1364    178.336      0.219      0.827      -313.581   391.854
VZ_Priority_Geo_c         -9.8730     21.564     -0.458      0.648       -52.522    32.776
Vision_Zero_c             -1.0776     42.143     -0.026      0.980       -84.430    82.275
Bikes_c                  -67.0717    125.179     -0.536      0.593      -314.654   180.511
High_Crash_Corridor_i     -9.4950    279.716     -0.034      0.973      -562.724   543.734
Pedestrian_Safety_i     -368.9591    377.113     -0.978      0.330     -1114.822   376.904
Senior_Safety_i         -536.8048   3842.233     -0.140      0.889     -8136.073  7062.463
Traffic_Calming_i       -450.2568    402.401     -1.119      0.265     -1246.136   345.622
Traffic_Network_Chng_i  1123.0468   1850.528      0.607      0.545     -2536.975  4783.069
VZ_Priority_Geo_i        -43.5684    434.432     -0.100      0.920      -902.799   815.662
Vision_Zero_i            -16.9920    560.100     -0.030      0.976     -1124.773  1090.789
==============================================================================
Omnibus:                      267.406   Durbin-Watson:                   0.990
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            34362.723
Skew:                           8.331   Prob(JB):                         0.00
Kurtosis:                      74.750   Cond. No.                         345.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [222]:
with_projects_area = with_projects.copy()

In [223]:
with_projects_area.loc[:,'High_Crash_Corridor_c':'Vision_Zero_i'] = with_projects_area.loc[:,'High_Crash_Corridor_c':'Vision_Zero_i'].apply(lambda x: x / with_projects_area.Shape_Area)

In [224]:
data3 = with_projects_area[with_projects_area.Total_Pop > 0][with_projects.Year > 2013] 
# one precinct was opened in 2013 and odes not have demog data
lm3 = smf.ols(eqt, data = data3).fit()

/opt/rh/anaconda/root/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [225]:
std_pop = with_projects.Total_Pop.std()
mean_pop = with_projects.Total_Pop.mean()
threshold = -2 * std_pop + mean_pop

In [226]:
# removes a handful of precincts with very low outlier populations (ie central park)
pre_VZ = with_projects[with_projects.Year < 2014][with_projects.Total_Pop > threshold]

/opt/rh/anaconda/root/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


In [227]:
# removes a handful of outlier precints with v low populations (ie Central Park)
data4 = with_projects[with_projects.Total_Pop > threshold]

In [228]:
lm4 = smf.ols(eqt, data = data4).fit()
lm4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         Per_Cap_Injury   R-squared:                       0.602
Model:                            OLS   Adj. R-squared:                  0.568
Method:                 Least Squares   F-statistic:                     17.40
Date:                Mon, 05 Dec 2016   Prob (F-statistic):           3.13e-32
Time:                        14:29:22   Log-Likelihood:                -1157.7
No. Observations:                 225   AIC:                             2351.
Df Residuals:                     207   BIC:                             2413.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Speed_Bump_Total           2.8653      0.588      4.873      0.000         1.706     4.025
Neigh_Slow_Zone_Total     -0.9954      8.201     -0.121      0.904       -17.164    15.173
High_Crash_Corridor_c      1.3689      0.712      1.923      0.056        -0.035     2.772
New_Public_Space_c        15.7026      6.254      2.511      0.013         3.373    28.032
Pedestrian_Safety_c       -0.3351      1.170     -0.286      0.775        -2.642     1.972
Senior_Safety_c           30.2584     25.478      1.188      0.236       -19.970    80.487
Traffic_Calming_c          0.2041      0.212      0.965      0.336        -0.213     0.621
Traffic_Network_Chng_c     3.2828      1.616      2.031      0.044         0.096     6.470
VZ_Priority_Geo_c          0.3575      0.195      1.837      0.068        -0.026     0.741
Vision_Zero_c              0.4542      0.379      1.197      0.233        -0.294     1.202
Bikes_c                    5.5432      1.142      4.855      0.000         3.292     7.794
High_Crash_Corridor_i     -1.5275      2.549     -0.599      0.550        -6.552     3.497
Pedestrian_Safety_i        3.8025      3.419      1.112      0.267        -2.938    10.543
Senior_Safety_i          -13.3780     33.987     -0.394      0.694       -80.382    53.626
Traffic_Calming_i          3.6801      3.653      1.007      0.315        -3.521    10.882
Traffic_Network_Chng_i     8.3814     16.801      0.499      0.618       -24.742    41.505
VZ_Priority_Geo_i         19.5932      3.933      4.982      0.000        11.839    27.347
Vision_Zero_i              9.4016      4.992      1.883      0.061        -0.441    19.244
==============================================================================
Omnibus:                       89.745   Durbin-Watson:                   0.770
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              494.539
Skew:                           1.460   Prob(JB):                    4.10e-108
Kurtosis:                       9.650   Cond. No.                         334.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [229]:
data5 = with_projects_area[with_projects_area.Total_Pop > threshold]

In [230]:
lm5 = smf.ols(eqt, data = data5).fit()
lm5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         Per_Cap_Injury   R-squared:                       0.648
Model:                            OLS   Adj. R-squared:                  0.618
Method:                 Least Squares   F-statistic:                     21.20
Date:                Mon, 05 Dec 2016   Prob (F-statistic):           1.61e-37
Time:                        14:29:22   Log-Likelihood:                -1143.8
No. Observations:                 225   AIC:                             2324.
Df Residuals:                     207   BIC:                             2385.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Speed_Bump_Total           3.3795      0.516      6.546      0.000         2.362     4.397
Neigh_Slow_Zone_Total      0.7589      7.645      0.099      0.921       -14.312    15.830
High_Crash_Corridor_c      3.6122      1.565      2.308      0.022         0.526     6.698
New_Public_Space_c        36.8724     13.841      2.664      0.008         9.585    64.160
Pedestrian_Safety_c       -1.0697      1.616     -0.662      0.509        -4.256     2.116
Senior_Safety_c           14.7863     23.756      0.622      0.534       -32.048    61.621
Traffic_Calming_c          1.6569      0.391      4.239      0.000         0.886     2.428
Traffic_Network_Chng_c     2.4515      2.742      0.894      0.372        -2.954     7.857
VZ_Priority_Geo_c          0.5370      0.388      1.384      0.168        -0.228     1.302
Vision_Zero_c             -0.1772      0.998     -0.178      0.859        -2.144     1.790
Bikes_c                    6.0711      1.263      4.809      0.000         3.582     8.560
High_Crash_Corridor_i     -3.4844      8.091     -0.431      0.667       -19.435    12.467
Pedestrian_Safety_i        5.0105      3.452      1.451      0.148        -1.796    11.817
Senior_Safety_i          -64.0357    183.447     -0.349      0.727      -425.700   297.629
Traffic_Calming_i         -4.6847      6.261     -0.748      0.455       -17.028     7.658
Traffic_Network_Chng_i    33.7132     24.851      1.357      0.176       -15.281    82.707
VZ_Priority_Geo_i         20.7257      4.373      4.739      0.000        12.104    29.347
Vision_Zero_i             14.0786      8.279      1.701      0.091        -2.243    30.400
==============================================================================
Omnibus:                       30.837   Durbin-Watson:                   0.812
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               61.620
Skew:                           0.685   Prob(JB):                     4.16e-14
Kurtosis:                       5.166   Cond. No.                         780.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [244]:
lm6 = smf.ols('Per_Cap_Injury ~ Speed_Bump_Total -1', data = data5).fit()
lm6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         Per_Cap_Injury   R-squared:                       0.308
Model:                            OLS   Adj. R-squared:                  0.305
Method:                 Least Squares   F-statistic:                     99.55
Date:                Mon, 05 Dec 2016   Prob (F-statistic):           1.24e-19
Time:                        14:34:30   Log-Likelihood:                -1220.0
No. Observations:                 225   AIC:                             2442.
Df Residuals:                     224   BIC:                             2445.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Speed_Bump_Total     5.4102      0.542      9.977      0.000         4.342     6.479
==============================================================================
Omnibus:                      116.532   Durbin-Watson:                   0.702
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1044.557
Skew:                           1.805   Prob(JB):                    1.50e-227
Kurtosis:                      12.919   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [239]:
merge2['PrecinctYear'] = merge2['PrecinctYear'].apply(lambda x: str(x).zfill(7))

In [247]:
remerge = pd.merge(merge2, precincts_shp, left_on = 'Precinct', right_on = 'Precinct', how = 'outer')

In [249]:
remerge.fillna(0, inplace = True)

In [252]:
remerge['Injuries_Per_Thou'] = remerge['NUMBER OF PERSONS INJURED'] / (remerge['Total_Pop'] / 1000)

In [258]:
remerge.drop('Per_Cap_Injury_Per_Thou', axis = 1)

PrecinctYear  Precinct  Year  Taxis  Flyers  Seniors  Outreach  HandsOn  \
0        0682013      68.0  2013      0       0        0         0        0   
1        0682014      68.0  2014      0       0        4         4        0   
2        0682015      68.0  2015      4       0        3        11        2   
3        1022013     102.0  2013      0       0        0         0        0   
4        1022014     102.0  2014      0       4        4         7        1   
5        1022015     102.0  2015      8       0        2         4        0   
6        0252013      25.0  2013      0       0        0         0        0   
7        0252014      25.0  2014      0       0        2        12        0   
8        0252015      25.0  2015      2       0        0         6        0   
9        0422013      42.0  2013      0       0        0         0        0   
10       0422014      42.0  2014      0       0        0        22        1   
11       0422015      42.0  2015      2       0        1        16        0   
12       0762013      76.0  2013      0       0        0         0        0   
13       0762014      76.0  2014      0       0        0         2        0   
14       0762015      76.0  2015      3       0        0         4        0   
15       1102013     110.0  2013      0       0        0         0        0   
16       1102014     110.0  2014      0       0        3         4        1   
17       1102015     110.0  2015      6       0        6        13        4   
18       0332013      33.0  2013      0       0        0         0        0   
19       0332014      33.0  2014      2       2        0        14        3   
20       0332015      33.0  2015      4       0        1        11        9   
21       0502013      50.0  2013      0       0        0         0        0   
22       0502014      50.0  2014      0       5        1         8        1   
23       0502015      50.0  2015      3       0        0         6        0   
24       0672013      67.0  2013      0       0        0         0        0   
25       0672014      67.0  2014      0       0        1        23        0   
26       0672015      67.0  2015      1       0        2        14        0   
27       0842013      84.0  2013      0       0        0         0        0   
28       0842014      84.0  2014      0       0        0         7        4   
29       0842015      84.0  2015      5       1        1        11        3   
..           ...       ...   ...    ...     ...      ...       ...      ...   
201      1202013     120.0  2013      0       0        0         0        0   
202      1202014     120.0  2014      0       1        1        19        4   
203      1202015     120.0  2015      6       0        1         8        1   
204      0092013       9.0  2013      0       0        0         0        0   
205      0092014       9.0  2014      0       0        0         5        0   
206      0092015       9.0  2015      0       0        2         0        0   
207      0262013      26.0  2013      0       0        0         0        0   
208      0262014      26.0  2014      1       4        0         5        1   
209      0262015      26.0  2015      0       0        0         6        0   
210      0432013      43.0  2013      0       0        0         0        0   
211      0432014      43.0  2014      0       3        3        21        3   
212      0432015      43.0  2015      0       0        1        17        1   
213      0602013      60.0  2013      0       0        0         0        0   
214      0602014      60.0  2014      0       0        4        12        1   
215      0602015      60.0  2015      0       0        1         1        3   
216      0772013      77.0  2013      0       0        0         0        0   
217      0772014      77.0  2014      0       1        0         7        1   
218      0772015      77.0  2015      7       0        0         7        0   
219      0942013      94.0  2013      0       0       

In [261]:
remerge.to_csv('mergedv3.csv')